In [1]:
'''
Code includes three functions: 
clean_data - 
create_model - 
predict_response - 
Logistic regression to classify cuisine from a string of ingredients
'''

'\nCode includes three functions: \nclean_data - \ncreate_model - \npredict_response - \nLogistic regression to classify cuisine from a string of ingredients\n'

### Import libraries

In [2]:
#nltk.download()
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import nltk
import re
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
#from sklearn.metrics import classification_report
#from sklearn import grid_search

import pickle
from sklearn.externals import joblib

### Function 1: import & clean data 

In [3]:
def clean_data(path):
    #import data
    df = pd.read_json(path)
    #clean data
    df['ingredients_clean_string'] = [' , '.join(z).strip() for z in df['ingredients']]  
    df['ingredients_string'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in df['ingredients']]       
    return df

In [4]:
df = clean_data("../data/train.json")

### Function 2: train model

In [5]:
def train_model(training_df):
    #instantiate vectorizer
    vect = TfidfVectorizer(stop_words = 'english', #remove standard english stop words
                           ngram_range = (1 , 1), #range of n-values for different n-grams to be extracted
                           analyzer = "word", #feature made of word (not character) n-grams
                           max_df = .57, #ignore terms that have a document frequency higher than threshold 
                           binary = False , 
                           token_pattern = r'\w+' , #what constitutes a “token”
                           sublinear_tf = False
                          )
    #use vectorizer on training corpus
    predictors = vect.fit_transform(df['ingredients_string']).todense()
    #define response
    response = df['cuisine']
    #instantiate model
    clf = LogisticRegression()
    #train the model
    parameters = {'C':[1, 10]}
    classifier = GridSearchCV(clf, parameters)
    classifier = classifier.fit(predictors, response) 
    # return vect, classifier

    #create pickle files
    joblib.dump(vect, 've.pkl') 
    joblib.dump(classifier, 'cl.pkl') 

In [6]:
train_model(df)

In [7]:
#vect, classifier = train_model(df)

### Function 3: predict cuisine for user inputted list

In [8]:
def predict_response(user_input):
    #load pickle vectorizer & classifier
    vect = joblib.load('ve.pkl')
    classifier = joblib.load('cl.pkl')
    
    #vectorize the input
    predictors_input = vect.transform([user_input])
    predictors_input.toarray()
    
    #predict response
    response_output = ''.join(classifier.predict(predictors_input))
    return response_output

In [16]:
predict_response('kung pao chicken')

'chinese'